<a href="https://colab.research.google.com/github/Raian-Rahman/Pytorch-Practice/blob/main/(ResNet_50)_Cifer10_Dataset_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import random_split,DataLoader
import torchvision.models  as models
from torch.optim import lr_scheduler
%matplotlib inline


### Loading the dataset

In [3]:
train_data = datasets.CIFAR10('data',train=True,download='True',transform=transforms.ToTensor())
print(len(train_data))
train,val = random_split(train_data,[45000,5000])
train_loader = DataLoader(train,batch_size = 32)
val_loader = DataLoader(val,batch_size=32)

Extracting data/cifar-10-python.tar.gz to data
50000


### Loading the pre-trained model

For this experiment we will use a pretrained model (currently Resnet-50). Now, most of the pretraned models are trained for a different number of models. So, what we can do is to change the last layer to customize it according to our need. 

In [10]:
resnet50 = models.resnet50(pretrained=True)

#to calculate the number of input features in different layers
num_input = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_input,10)
resnet50.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Defining a Loss Function and Optimizer

In [16]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(resnet50.parameters(),lr=0.01)
loss = nn.CrossEntropyLoss()

#learning rate scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimiser,step_size=7,gamma=0.1)


In [17]:
# define training loop

nb_epochs = 15

for epoch in range(nb_epochs):
    losses1 = []
    resnet50.train()
    for batch in train_loader:
        x ,y = batch
        # x : batch_size * 1_channel * 28 * 28
        #b = x.size(0)
        #x = x.view(b,-1).cuda()
        #forward pass
        x = x.cuda()
        l = resnet50(x) # l stands for logit
        
        #compute the loss fucnction
        J = loss(l,y.cuda())
        
        #cleaning the gradient 
        optimiser.zero_grad()
        #params.grad._zero()
       
        #compute accumulatinfg the partial gradient 
        J.backward() #params.
        #params.grad.add_(dJ/dparams)
        
        #update the parameters
        optimiser.step()
        #with torch.no_grad(): params = params - alpha*params.grad
        losses1.append(J.item())
    losses = []
    accuracies = []
    resnet50.eval()
    step_lr_scheduler.step()
    for batch in val_loader:
        x ,y = batch
        # x : batch_size * 1_channel * 28 * 28
        x = x.cuda()
        #forward pass
        with torch.no_grad():
            l = resnet50(x) # l stands for logit
            #compute the loss fucnction
            J = loss(l,y.cuda())
            losses.append(J.item())
            accuracies.append(y.cuda().eq(l.detach().argmax(dim=1)).float().mean())
    print(f'Epoch {epoch+1},train loss: {torch.tensor(losses1).mean():.2f}  validation loss: {torch.tensor(losses).mean():.2f}')
    print(f'Epoch {epoch+1},Accuracy: {torch.tensor(accuracies).mean():.5f}')

Epoch 1,train loss: 0.06  validation loss: 0.59
Epoch 1,Accuracy: 0.84773
Epoch 2,train loss: 0.01  validation loss: 0.69
Epoch 2,Accuracy: 0.85092
Epoch 3,train loss: 0.00  validation loss: 0.77
Epoch 3,Accuracy: 0.85191
Epoch 4,train loss: 0.00  validation loss: 0.83
Epoch 4,Accuracy: 0.85151
Epoch 5,train loss: 0.00  validation loss: 0.87
Epoch 5,Accuracy: 0.85111
Epoch 6,train loss: 0.00  validation loss: 0.91
Epoch 6,Accuracy: 0.85131
Epoch 7,train loss: 0.00  validation loss: 0.94
Epoch 7,Accuracy: 0.85151
Epoch 8,train loss: 0.00  validation loss: 0.94
Epoch 8,Accuracy: 0.85151
Epoch 9,train loss: 0.00  validation loss: 0.94
Epoch 9,Accuracy: 0.85151
Epoch 10,train loss: 0.00  validation loss: 0.95
Epoch 10,Accuracy: 0.85151
Epoch 11,train loss: 0.00  validation loss: 0.95
Epoch 11,Accuracy: 0.85151
Epoch 12,train loss: 0.00  validation loss: 0.95
Epoch 12,Accuracy: 0.85151
Epoch 13,train loss: 0.00  validation loss: 0.95
Epoch 13,Accuracy: 0.85151
Epoch 14,train loss: 0.00  val

```
Epoch 1,train loss: 2.30  validation loss: 2.30
Epoch 1,Accuracy: 0.11525
Epoch 2,train loss: 2.30  validation loss: 2.29
Epoch 2,Accuracy: 0.14291
Epoch 3,train loss: 2.24  validation loss: 2.16
Epoch 3,Accuracy: 0.22552
Epoch 4,train loss: 2.08  validation loss: 2.00
Epoch 4,Accuracy: 0.26931
Epoch 5,train loss: 1.93  validation loss: 2.17
Epoch 5,Accuracy: 0.23865
Epoch 6,train loss: 1.82  validation loss: 1.96
Epoch 6,Accuracy: 0.30912
Epoch 7,train loss: 1.72  validation loss: 1.86
Epoch 7,Accuracy: 0.36704
Epoch 8,train loss: 1.62  validation loss: 1.73
Epoch 8,Accuracy: 0.42456
Epoch 9,train loss: 1.56  validation loss: 1.61
Epoch 9,Accuracy: 0.46377
Epoch 10,train loss: 1.50  validation loss: 1.60
Epoch 10,Accuracy: 0.47452
Epoch 11,train loss: 1.46  validation loss: 1.54
Epoch 11,Accuracy: 0.49303
Epoch 12,train loss: 1.42  validation loss: 1.50
Epoch 12,Accuracy: 0.51791
Epoch 13,train loss: 1.38  validation loss: 1.50
Epoch 13,Accuracy: 0.51990
Epoch 14,train loss: 1.35  validation loss: 1.50
Epoch 14,Accuracy: 0.52189
Epoch 15,train loss: 1.32  validation loss: 1.50
Epoch 15,Accuracy: 0.52428
Epoch 16,train loss: 1.29  validation loss: 1.66
Epoch 16,Accuracy: 0.46477
Epoch 17,train loss: 1.23  validation loss: 1.47
Epoch 17,Accuracy: 0.53145
Epoch 18,train loss: 1.16  validation loss: 1.48
Epoch 18,Accuracy: 0.52727
Epoch 19,train loss: 1.13  validation loss: 1.47
Epoch 19,Accuracy: 0.53662
Epoch 20,train loss: 1.10  validation loss: 1.51
Epoch 20,Accuracy: 0.52946
Epoch 21,train loss: 1.08  validation loss: 1.49
Epoch 21,Accuracy: 0.52866
Epoch 22,train loss: 1.05  validation loss: 1.50
Epoch 22,Accuracy: 0.53185
Epoch 23,train loss: 1.03  validation loss: 1.54
Epoch 23,Accuracy: 0.53702
Epoch 24,train loss: 1.01  validation loss: 1.56
Epoch 24,Accuracy: 0.53304
Epoch 25,train loss: 0.99  validation loss: 1.62
Epoch 25,Accuracy: 0.52687
Epoch 26,train loss: 0.97  validation loss: 1.64
Epoch 26,Accuracy: 0.53025
Epoch 27,train loss: 0.95  validation loss: 1.62
Epoch 27,Accuracy: 0.54061
Epoch 28,train loss: 0.94  validation loss: 1.68
Epoch 28,Accuracy: 0.52767
Epoch 29,train loss: 0.92  validation loss: 1.65
Epoch 29,Accuracy: 0.52767
Epoch 30,train loss: 0.91  validation loss: 1.68
Epoch 30,Accuracy: 0.53105
Epoch 31,train loss: 0.89  validation loss: 1.73
Epoch 31,Accuracy: 0.52408
Epoch 32,train loss: 0.87  validation loss: 1.74
Epoch 32,Accuracy: 0.52767
Epoch 33,train loss: 0.86  validation loss: 1.72
Epoch 33,Accuracy: 0.53682
Epoch 34,train loss: 0.85  validation loss: 1.79
Epoch 34,Accuracy: 0.52588
Epoch 35,train loss: 0.84  validation loss: 1.76
Epoch 35,Accuracy: 0.53145
Epoch 36,train loss: 0.75  validation loss: 1.79
Epoch 36,Accuracy: 0.55792
Epoch 37,train loss: 0.70  validation loss: 1.82
Epoch 37,Accuracy: 0.54558
Epoch 38,train loss: 0.67  validation loss: 1.73
Epoch 38,Accuracy: 0.55852
Epoch 39,train loss: 0.66  validation loss: 1.79
Epoch 39,Accuracy: 0.55872
Epoch 40,train loss: 0.64  validation loss: 1.80
Epoch 40,Accuracy: 0.55613
```